# Science experiment

I want to try technique.

- Nothing. 4d tetrode. -> MSE decode.
- PCA. 3d tetrode. -> MSE decode.
- Autoencoder. 3d tetrode. -> MSE decode.

For each technique, I need to do the following:
- get the spikes in the right format
- transform properly
- fit model
- predict model
- MSE on model

None of this is realtime.

In [1]:
import neural_dimensionality_reduction as ndr

In [2]:
import numpy as np
from replay_trajectory_classification import ClusterlessDecoder

model_kwargs = {
    'bandwidth': np.array([1.0, 1.0, 1.0, 1.0, 12.5]) # amplitude 1, amplitude 2, amplitude 3, amplitude 4, position
    }

## Step 1: Just do tetrode (d=4) no reduction

In [3]:
data1, position = ndr.get_formatted_data(ndr.data_dir)

/nix/store/0m459gzwib59dgahny5wrpwk0v1g17sq-python3-3.7.6-env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (11,12,13,15) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [4]:
data1.shape

(615851, 4, 1)

In [5]:
position.shape

(615851,)

In [6]:
decoder1 = ClusterlessDecoder(model_kwargs=model_kwargs)

In [7]:
decoder1.fit(position, data1)

/home/trevortknguyen/repos/dimensionality/replay_trajectory_classification/state_transition.py:10: RuntimeWarning: invalid value encountered in true_divide
  x /= x.sum(axis=1, keepdims=True)


ClusterlessDecoder(infer_track_interior=True,
                   initial_conditions_type='uniform_on_track',
                   model=<class 'replay_trajectory_classification.misc.NumbaKDE'>,
                   model_kwargs={'bandwidth': array([ 1. ,  1. ,  1. ,  1. , 12.5])},
                   movement_var=0.05,
                   occupancy_kwargs={'bandwidth': array([ 1. ,  1. ,  1. ,  1. , 12.5])},
                   occupancy_model=<class 'replay_trajectory_classification.misc.NumbaKDE'>,
                   place_bin_size=2.0, position_range=None, replay_speed=40,
                   transition_type='random_walk')

In [8]:
time_ind = slice(0, 5000)
time = np.linspace(0, 1, data1.shape[0])

In [9]:
results1 = decoder1.predict(data1[time_ind], time=time[time_ind])
results1

<xarray.Dataset>
Dimensions:            (position: 172, time: 5000)
Coordinates:
  * time               (time) float64 0.0 1.624e-06 ... 0.008116 0.008117
  * position           (position) float64 0.9966 2.99 4.983 ... 339.8 341.8
Data variables:
    likelihood         (time, position) float64 0.9854 0.9854 ... 0.9953 0.9954
    causal_posterior   (time, position) float64 0.006352 0.006352 ... 2.699e-67
    acausal_posterior  (time, position) float64 7.815e-16 ... 2.699e-67

In [11]:
ndr.calculate_mse_causal(results1, position, time_ind)

3911.163288955438

## Step 2: Do PCA reduction

In [12]:
data2, position = ndr.get_formatted_data(ndr.)).data_dir)

/nix/store/0m459gzwib59dgahny5wrpwk0v1g17sq-python3-3.7.6-env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (11,12,13,15) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [13]:
na = data2.reshape((data2.shape[0], 4,))

In [14]:
spikes_bool = np.logical_not(np.isnan(na)).any(axis=1)
spikes_idx = np.argwhere(spikes_bool)

In [15]:
spikes_idx = spikes_idx.reshape((spikes_idx.shape[0]))

In [16]:
na[spikes_idx].shape

(62003, 4)

In [17]:
transformed = ndr.transform_spikes_to_marks(na[spikes_idx])

In [18]:
transformed_good = np.empty((data2.shape[0],3,1))
transformed_good[:] = np.nan
transformed_good[:,:,0][spikes_idx] = transformed

In [20]:
decoder2 = ClusterlessDecoder(model_kwargs=model_kwargs)

In [21]:
decoder2.fit(position, transformed_good)

/home/trevortknguyen/repos/dimensionality/replay_trajectory_classification/state_transition.py:10: RuntimeWarning: invalid value encountered in true_divide
  x /= x.sum(axis=1, keepdims=True)


ClusterlessDecoder(infer_track_interior=True,
                   initial_conditions_type='uniform_on_track',
                   model=<class 'replay_trajectory_classification.misc.NumbaKDE'>,
                   model_kwargs={'bandwidth': array([ 1. ,  1. ,  1. ,  1. , 12.5])},
                   movement_var=0.05,
                   occupancy_kwargs={'bandwidth': array([ 1. ,  1. ,  1. ,  1. , 12.5])},
                   occupancy_model=<class 'replay_trajectory_classification.misc.NumbaKDE'>,
                   place_bin_size=2.0, position_range=None, replay_speed=40,
                   transition_type='random_walk')

In [22]:
time_ind = slice(0, 5000)
time = np.linspace(0, 1, transformed_good.shape[0])

In [23]:
results = decoder2.predict(transformed_good[time_ind], time=time[time_ind])
results

<xarray.Dataset>
Dimensions:            (position: 172, time: 5000)
Coordinates:
  * time               (time) float64 0.0 1.624e-06 ... 0.008116 0.008117
  * position           (position) float64 0.9966 2.99 4.983 ... 339.8 341.8
Data variables:
    likelihood         (time, position) float64 0.9854 0.9854 ... 0.9953 0.9954
    causal_posterior   (time, position) float64 0.006352 0.006352 ... 0.04214
    acausal_posterior  (time, position) float64 4.335e-11 5.679e-11 ... 0.04214

In [24]:
ndr.calculate_mse_causal(results, position, time_ind)

1665.1166659955381

## Step 3: Do autoencoder reduction

In [26]:
data3, position = ndr.get_formatted_data(ndr.data_dir)

/nix/store/0m459gzwib59dgahny5wrpwk0v1g17sq-python3-3.7.6-env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (11,12,13,15) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [27]:
na = data3.reshape((data3.shape[0], 4,))

In [28]:
spikes_bool = np.logical_not(np.isnan(na)).any(axis=1)
spikes_idx = np.argwhere(spikes_bool)
spikes_idx = spikes_idx.reshape((spikes_idx.shape[0]))

In [33]:
spikes_only = na[spikes_idx]

In [34]:
import torch

In [35]:
x_train_torch = torch.from_numpy(spikes_only)

In [37]:
D_in = D_out = 4
H = 3

In [38]:
class Autoencoder(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(Autoencoder, self).__init__()
        self.fc1 = torch.nn.Linear(D_in, H)
        self.fc2 = torch.nn.LeakyReLU()
        self.fc3 = torch.nn.Linear(H, D_out)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

    def dim_reduce(self, x):
        with torch.no_grad():
            x = self.fc1(x)
            return x

In [39]:
model = Autoencoder(D_in, H, D_out)

In [42]:
from math import ceil
loss_fn = torch.nn.MSELoss()

learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epochs = 50
batch_size = 512

examples, _ = x_train_torch.shape

for e in range(epochs):
    for t in range(ceil(examples/batch_size)):
        x_slice = x_train_torch[t * batch_size : (t+1) * batch_size]

        y_pred = model(x_slice)

        loss = loss_fn(y_pred, x_slice)
        if t % 100 == 99:
            print(t, loss.item())

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

99 17896.86328125
99 17686.0
99 17298.318359375
99 16855.576171875
99 16380.580078125
99 15873.697265625
99 15337.7958984375
99 14778.1748046875
99 14200.734375
99 13611.6181640625
99 13016.8896484375
99 12320.671875
99 11365.453125
99 10560.802734375
99 9872.921875
99 9274.109375
99 8739.732421875
99 8248.6171875
99 7784.8984375
99 7338.5693359375
99 6903.5126953125
99 6475.4814453125
99 6051.51416015625
99 5630.19091796875
99 5211.78271484375
99 4798.01953125
99 4391.64892578125
99 3996.026611328125
99 3614.762939453125
99 3251.40185546875
99 2909.149658203125
99 2590.657958984375
99 2297.84912109375
99 2031.826416015625
99 1792.8397216796875
99 1580.3270263671875
99 1393.021240234375
99 1229.1021728515625
99 1086.39013671875
99 962.5504150390625
99 855.2749633789062
99 762.4345092773438
99 682.1687622070312
99 612.9224853515625
99 553.4224243164062
99 502.62615966796875
99 459.6519470214844
99 423.7140808105469
99 394.07110595703125
99 369.9969482421875


In [43]:
xs_result = model.dim_reduce(x_train_torch).numpy()

In [46]:
transformed_good = np.empty((data2.shape[0],3,1))
transformed_good[:] = np.nan
transformed_good[:,:,0][spikes_idx] = xs_result

In [49]:
decoder3 = ClusterlessDecoder(model_kwargs=model_kwargs)

In [50]:
decoder3.fit(position, transformed_good)

/home/trevortknguyen/repos/dimensionality/replay_trajectory_classification/state_transition.py:10: RuntimeWarning: invalid value encountered in true_divide
  x /= x.sum(axis=1, keepdims=True)


ClusterlessDecoder(infer_track_interior=True,
                   initial_conditions_type='uniform_on_track',
                   model=<class 'replay_trajectory_classification.misc.NumbaKDE'>,
                   model_kwargs={'bandwidth': array([ 1. ,  1. ,  1. ,  1. , 12.5])},
                   movement_var=0.05,
                   occupancy_kwargs={'bandwidth': array([ 1. ,  1. ,  1. ,  1. , 12.5])},
                   occupancy_model=<class 'replay_trajectory_classification.misc.NumbaKDE'>,
                   place_bin_size=2.0, position_range=None, replay_speed=40,
                   transition_type='random_walk')

In [51]:
time_ind = slice(0, 5000)
time = np.linspace(0, 1, transformed_good.shape[0])

In [53]:
results3 = decoder3.predict(transformed_good[time_ind], time=time[time_ind])
results3

<xarray.Dataset>
Dimensions:            (position: 172, time: 5000)
Coordinates:
  * time               (time) float64 0.0 1.624e-06 ... 0.008116 0.008117
  * position           (position) float64 0.9966 2.99 4.983 ... 339.8 341.8
Data variables:
    likelihood         (time, position) float64 0.9854 0.9854 ... 0.9953 0.9954
    causal_posterior   (time, position) float64 0.006352 0.006352 ... 4.879e-44
    acausal_posterior  (time, position) float64 2.096e-11 ... 4.879e-44

In [54]:
ndr.calculate_mse_causal(results3, position, time_ind)

4860.100245455682